In [1]:
import os
import sys
os.chdir('../')
sys.path.append('../')

import torch
from utils.utils import train,prepare,evaluate,tune
from models.Interactors import FIM_Interactor, KNRM_Interactor
from models.Encoders.FIM import FIM_Encoder
from models.SFI import SFI_gating, SFI_gating_MultiView
from configs.ManualConfig import hparams

In [2]:
name='sfi'
hparams['k'] = 30
hparams['his_size'] = 80
hparams['select'] = 'gating'
hparams['onehot'] = True
hparams['device'] = 'cuda:0'
# hparams['threshold'] = 0.5

In [3]:
# hparams['validate'] = True
vocab, loaders = prepare(hparams, pin_memory=False)

[2021-04-21 16:39:09,167] INFO (root) Hyper Parameters are
{'scale': 'demo', 'mode': 'train', 'batch_size': 10, 'title_size': 20, 'abs_size': 40, 'his_size': 50, 'vert_num': 18, 'subvert_num': 293, 'npratio': 4, 'dropout_p': 0.2, 'query_dim': 200, 'embedding_dim': 300, 'filter_num': 150, 'value_dim': 16, 'head_num': 16, 'epochs': 8, 'metrics': 'auc,mean_mrr,ndcg@5,ndcg@10', 'device': 'cuda:0', 'attrs': ['title'], 'k': 30, 'select': 'gating', 'save_step': [0], 'news_id': False, 'validate': False, 'interval': 10, 'spadam': True, 'onehot': True, 'val_freq': 1, 'schedule': None}
[2021-04-21 16:39:09,168] INFO (root) preparing dataset...
[2021-04-21 16:39:13,049] INFO (torchtext.vocab) Loading vectors from .vector_cache/glove.840B.300d.txt.pt


In [4]:
record = next(iter(loaders[0]))

In [21]:
encoder = FIM_Encoder(hparams, vocab)
# interactor = KNRM_Interactor()

hparams['name'] = '-'.join([name,encoder.name,hparams['select']])

# sfi = SFI_gating_MultiView(hparams, encoder, interactor).to(hparams['device'])
sfi = SFI_gating(hparams, encoder).to(hparams['device'])

# sfi.load_state_dict(torch.load('/home/peitian_zhang/Codes/News-Recommendation/data/model_params/sfi-fim-fim-gating/large_epoch4_step33832_[hs=50,topk=30,attrs=title].model', map_location=hparams['device'])['model'])

In [22]:
a = sfi.selectionProject.weight.clone().detach()

In [23]:
a

tensor([[ 0.0158,  0.0521, -0.0074,  ...,  0.0379, -0.0240, -0.0457],
        [-0.0931, -0.0150,  0.0645,  ...,  0.0029,  0.0418,  0.0092],
        [-0.0204, -0.0933, -0.0630,  ...,  0.0016,  0.0417,  0.0152],
        ...,
        [-0.1802,  0.1111,  0.0246,  ...,  0.0459, -0.1516, -0.0268],
        [ 0.0046, -0.1251,  0.0104,  ...,  0.1005, -0.2690,  0.0798],
        [-0.0408, -0.1457,  0.0876,  ...,  0.0576, -0.0006,  0.0653]],
       device='cuda:0')

In [24]:
hparams['epochs'] = 1
train(sfi, hparams, loaders)

[2021-04-21 16:46:49,945] INFO (root) training...
epoch 1 , step 290 , loss: 1.5595: 100%|██████████| 295/295 [00:22<00:00, 12.86it/s]
[2021-04-21 16:47:13,793] INFO (root) saved model of step 0, epoch 1 at data/model_params/sfi-fim-gating/demo_epoch1_step0_[hs=50,topk=30,attrs=title].model


SFI_gating(
  (encoder): FIM_Encoder(
    (embedding): Embedding(54316, 300, sparse=True)
    (ReLU): ReLU()
    (LayerNorm): LayerNorm((150,), eps=1e-05, elementwise_affine=True)
    (DropOut): Dropout(p=0.2, inplace=False)
    (CNN_d1): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(1,))
    (CNN_d2): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (CNN_d3): Conv1d(300, 150, kernel_size=(3,), stride=(1,), padding=(3,), dilation=(3,))
  )
  (softmax): Softmax(dim=-1)
  (interactor): FIM_Interactor(
    (SeqCNN3D): Sequential(
      (0): Conv3d(3, 32, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
      (1): ReLU()
      (2): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
      (3): Conv3d(32, 16, kernel_size=[3, 3, 3], stride=(1, 1, 1), padding=(1, 1, 1))
      (4): ReLU()
      (5): MaxPool3d(kernel_size=[3, 3, 3], stride=[3, 3, 3], padding=0, dilation=1, ceil_mode=False)
    )
  )
  (

In [25]:
c= a==sfi.selectionProject.weight

In [26]:
sfi.selectionProject.weight

Parameter containing:
tensor([[ 0.0158,  0.0521, -0.0074,  ...,  0.0379, -0.0240, -0.0457],
        [-0.0931, -0.0150,  0.0645,  ...,  0.0029,  0.0418,  0.0092],
        [-0.0204, -0.0933, -0.0630,  ...,  0.0016,  0.0417,  0.0152],
        ...,
        [-0.1802,  0.1111,  0.0246,  ...,  0.0459, -0.1516, -0.0268],
        [ 0.0046, -0.1251,  0.0104,  ...,  0.1005, -0.2690,  0.0798],
        [-0.0408, -0.1457,  0.0876,  ...,  0.0576, -0.0006,  0.0653]],
       device='cuda:0', requires_grad=True)

In [27]:
a[c==False]

tensor([], device='cuda:0')